In [1]:
import pandas as pd
import numpy as np
import os

# Load data

In [2]:
exclusions =  [
    'pilot_jb',
    'pilot_py',
    'pilot_st',
    'pilot_ah',
    'P101', #recorded over rosbag
    'P108', #turned off computer
    'P106', #only played 3 games
    'P174' #nao_action node died in game5
]

In [3]:
## FILE NAMES
DATA_DIR = "../data"
q_csv = "SpaceInvaders_NaoFeedback_September 7, 2022_09.43.csv"
conditions_csv = 'ParticipantURLs - Sheet1.csv'

## LOAD SURVEY DATA AND CONDITIONS
# load qualtrics csv
qdf = pd.read_csv(os.path.join(DATA_DIR,q_csv), header=[0], skiprows=[1,2])
# remove pilot participants (no ID)
qdf = qdf.dropna(subset=['ID'], how='any')
# remove extra columns
qdf = qdf.drop(['PROLIFIC_PID','STUDY_ID','SESSION_ID','RecipientLastName','RecipientFirstName','RecipientEmail','ExternalReference'], axis=1)

# remove excluded participants (including pilot)
qdf = qdf[~qdf.ID.isin(exclusions)]

# load condition csv and merge condition into qdf
condition_df = pd.read_csv(os.path.join(DATA_DIR,conditions_csv), header=1, usecols=['ID','Condition'])
qdf = qdf.merge(condition_df,on='ID', how='left')

# add condition info
condition_mapping = {'Condition':['A','B','C','D'],'Framing':['We','I','We','I'],'Timing':['Before','Before','After','After']}
cm_df = pd.DataFrame(data=condition_mapping)
qdf = qdf.merge(cm_df,on='Condition', how='left')

print(f"Loaded {qdf.shape[0]} participants")

#print(qdf['ID'])

Loaded 72 participants


In [34]:
qdf.columns

Index(['StartDate', 'EndDate', 'Status', 'IPAddress', 'Progress',
       'Duration (in seconds)', 'Finished', 'RecordedDate', 'ResponseId',
       'LocationLatitude',
       ...
       'Framing', 'Timing', 'FEEDBACK_SELECTWHEN_1', 'FEEDBACK_SELECTWHEN_2',
       'FEEDBACK_SELECTWHEN_3', 'FEEDBACK_SELECTWHEN_4',
       'FEEDBACK_SELECTWHEN_5', 'FEEDBACK_SELECTWHEN_6',
       'FEEDBACK_SELECTWHEN_7', 'FEEDBACK_SELECTWHEN_8'],
      dtype='object', length=148)

In [4]:
# categorical questions
qdf['GENDER'] = qdf['GENDER'].replace([1,2,3,4],['1_Male','2_Female','3_Nonbinary','4_PreferNotToSay'])
qdf['STUD'] = qdf['STUD'].replace([1,2],['1_Yes','2_No'])
qdf['SI'] = qdf['SI'].replace([1,2,3],['1_Yes','2_No','3_NotSure'])

qdf['RU_UNUS'] = qdf['RU_UNUS'].replace([1,2,3],['1_Yes','2_No','3_NotSure'])
qdf['RE_UNUS'] = qdf['RE_UNUS'].replace([1,2,3],['1_Yes','2_No','3_NotSure'])
qdf['RL_UNUS'] = qdf['RL_UNUS'].replace([1,2,3],['1_Yes','2_No','3_NotSure'])

qdf['RE_MANIP_TIMING'] = qdf['RE_MANIP_TIMING'].replace([1,2,3,4],['1_DontRememberReminder','2_Before','3_After','4_DontRememberOrdering'])

qdf['BEH_PREF'] = qdf['BEH_PREF'].replace([1,2,3,4],['1_Uncoop','2_Early','3_Late','4_NoPref'])


## Gender splits
1 = Male

2 = Female

3 = Nonbinary

4 = Prefer not to say

In [5]:
qdf.groupby('GENDER')['GENDER'].count()

GENDER
1_Male      34
2_Female    38
Name: GENDER, dtype: int64

In [6]:
pd.crosstab(qdf['Condition'],qdf['GENDER'])
#pd.crosstab(qdf['GENDER'], [qdf['Timing'], qdf['Framing']], rownames=['Gender'], colnames=['Timing', 'Framing'])


GENDER,1_Male,2_Female
Condition,,
A,9,9
B,8,10
C,8,10
D,9,9


In [7]:
pd.crosstab(qdf['Framing'],qdf['GENDER'])

GENDER,1_Male,2_Female
Framing,,
I,17,19
We,17,19


In [8]:
pd.crosstab(qdf['Timing'],qdf['GENDER'])

GENDER,1_Male,2_Female
Timing,,
After,17,19
Before,17,19


## Age info

In [9]:
qdf['AGE'].describe()

count    72.000000
mean     24.722222
std       6.699216
min      18.000000
25%      20.000000
50%      23.000000
75%      27.000000
max      53.000000
Name: AGE, dtype: float64

In [10]:
qdf.groupby("Condition")['AGE'].describe()[['mean','std']]

,mean,std
Condition,,
A,24.833333,6.166799
B,23.777778,6.112240
C,23.777778,3.573687
D,26.500000,9.696876


In [11]:
qdf.groupby("Framing")['AGE'].describe()

,count,mean,std,min,25%,50%,75%,max
Framing,,,,,,,,
I,36.0,25.138889,8.106973,18.0,19.75,23.0,27.25,53.0
We,36.0,24.305556,4.996110,18.0,20.00,23.0,26.25,42.0


In [12]:
qdf.groupby("Timing")['AGE'].describe()

,count,mean,std,min,25%,50%,75%,max
Timing,,,,,,,,
After,36.0,25.138889,7.333496,19.0,20.75,23.0,26.5,53.0
Before,36.0,24.305556,6.074864,18.0,20.00,23.0,27.0,44.0


## Other dem

Have you ever played the video game Space Invaders before?

In [13]:
pd.crosstab(qdf['Condition'],qdf['SI'])

SI,1_Yes,2_No,3_NotSure
Condition,,,
A,4,10,4
B,4,11,3
C,3,6,9
D,5,8,5


In [14]:
pd.crosstab(qdf['Framing'],qdf['SI'])

SI,1_Yes,2_No,3_NotSure
Framing,,,
I,9,19,8
We,7,16,13


In [15]:
pd.crosstab(qdf['Timing'],qdf['SI'])

SI,1_Yes,2_No,3_NotSure
Timing,,,
After,8,14,14
Before,8,21,7


**How often do you... interact with robots?**

1: Daily

2: 4-6 times a week

3: 2-3 times a week

4: Once a week

5: Once a month

6: Less than once a month

In [16]:
qdf.groupby("Condition")['OFTEN_3_R'].describe()

,count,mean,std,min,25%,50%,75%,max
Condition,,,,,,,,
A,18.0,5.277778,1.601674,1.0,5.25,6.0,6.0,6.0
B,18.0,5.388889,1.036901,3.0,5.00,6.0,6.0,6.0
C,18.0,4.555556,1.822158,1.0,3.25,5.5,6.0,6.0
D,18.0,4.944444,1.924218,1.0,4.50,6.0,6.0,6.0


In [17]:
qdf['OFTEN_2_V'].describe()

count    72.000000
mean      4.222222
std       1.654888
min       1.000000
25%       3.000000
50%       4.000000
75%       6.000000
max       6.000000
Name: OFTEN_2_V, dtype: float64

In [18]:
qdf['OFTEN_3_R'].describe()

count    72.000000
mean      5.041667
std       1.631015
min       1.000000
25%       5.000000
50%       6.000000
75%       6.000000
max       6.000000
Name: OFTEN_3_R, dtype: float64

In [19]:
qdf.groupby(['OFTEN_3_R']).size()/72

OFTEN_3_R
1    0.097222
2    0.013889
3    0.055556
4    0.069444
5    0.111111
6    0.652778
dtype: float64

In [20]:
qdf.groupby(['SI']).size()/72

SI
1_Yes        0.222222
2_No         0.486111
3_NotSure    0.291667
dtype: float64

In [21]:
qdf['OFTEN_1_C'].describe()

count    72.000000
mean      1.083333
std       0.496466
min       1.000000
25%       1.000000
50%       1.000000
75%       1.000000
max       5.000000
Name: OFTEN_1_C, dtype: float64

## Manipulation checks: Framing

In [22]:
qdf.groupby("Framing")['ROBOT_ORIENTATION'].describe()

,count,mean,std,min,25%,50%,75%,max
Framing,,,,,,,,
I,36.0,3.611111,1.399546,2.0,2.75,3.0,4.0,7.0
We,36.0,4.694444,1.450506,1.0,3.75,5.0,6.0,7.0


**Please rate the frequency of the following statements.**
From 1 (Never) to 7 (Always)

The robot reference the *team*

The robot reference *itself*

In [23]:
qdf.groupby("Framing")['REF_FREQ_SELF'].describe()

,count,mean,std,min,25%,50%,75%,max
Framing,,,,,,,,
I,36.0,4.611111,1.535816,1.0,4.0,4.0,6.0,7.0
We,36.0,2.555556,1.520234,1.0,1.0,2.5,4.0,7.0


In [24]:
qdf.groupby("Framing")['REF_FREQ_TEAM'].describe()

,count,mean,std,min,25%,50%,75%,max
Framing,,,,,,,,
I,36.0,2.777778,1.742102,1.0,1.00,2.0,4.0,7.0
We,36.0,4.750000,1.662614,1.0,3.75,5.0,6.0,7.0


**Please rate the statement on a scale from strongly disagree to strongly agree depending on how much you agree with them.**

The robot cares about the team

In [25]:
qdf.groupby("Framing")['ROBOT_CARES_TEAM_1'].describe()

,count,mean,std,min,25%,50%,75%,max
Framing,,,,,,,,
I,36.0,5.277778,1.649435,1.0,5.0,6.0,6.0,7.0
We,36.0,5.444444,1.423164,1.0,5.0,6.0,6.0,7.0


## Manipulation checks: Timing

**In the last two games, the robot reminded you to give feedback. When did the robot remind you to give feedback?**

1: I don't remember the robot reminding me to give feedback

2: Before the robot said that it was destroying enemies on the left side of the game screen

3: After the robot said that it was destroying enemies on the left side of the game screen

4: I don't remember the ordering

In [26]:
pd.crosstab(qdf['Timing'],qdf['RE_MANIP_TIMING'])/36

RE_MANIP_TIMING,2_Before,3_After,4_DontRememberOrdering
Timing,,,
After,0.111111,0.75,0.138889
Before,0.694444,0.25,0.055556


**Please rate the statements on a scale from strongly disagree to strongly agree depending on how much you agree with them.**

It was *distracting* to give feedback with the up/down arrow keys

It was *difficult* to give feedback with the up/down arrow keys

I think I was able to give the robot *helpful* feedback

In [27]:
qdf.groupby("Timing")[['FEEDBACK_AGREE_DISTRACTING','FEEDBACK_AGREE_DIFFICULT','FEEDBACK_AGREE_HELPFUL']].mean()


,FEEDBACK_AGREE_DISTRACTING,FEEDBACK_AGREE_DIFFICULT,FEEDBACK_AGREE_HELPFUL
Timing,,,
After,3.722222,2.388889,5.361111
Before,3.222222,2.138889,5.250000


**Please rate the statements on a scale from strongly disagree to strongly agree depending on how much you agree with them.**

I gave feedback to *signal the robot should change* its behavior

I gave feedback *in response to the robot changing* its behavior

In [28]:
qdf.groupby("Timing")[['FEEDBACK_AGREE_SIGNAL','FEEDBACK_AGREE_RESPONSE']].mean()


,FEEDBACK_AGREE_SIGNAL,FEEDBACK_AGREE_RESPONSE
Timing,,
After,4.527778,5.444444
Before,4.805556,4.972222


In [29]:
#qdf.to_csv("qualtrics_hri2023.csv")

# When

In [30]:
qdf['FEEDBACK_SELECTWHEN_1']= '1' in qdf['FEEDBACK_SELECTWHEN']

In [31]:
qdf['FEEDBACK_SELECTWHEN']

0           4,6,7
1         1,4,5,6
2           4,5,6
3       2,3,4,6,7
4     3,4,5,6,7,8
         ...     
67      3,4,5,6,7
68      3,4,5,6,7
69        1,4,5,6
70          4,6,7
71      3,4,5,6,7
Name: FEEDBACK_SELECTWHEN, Length: 72, dtype: object

In [32]:
qdf['FEEDBACK_SELECTWHEN_1'] = qdf['FEEDBACK_SELECTWHEN'].str.contains('1')
qdf['FEEDBACK_SELECTWHEN_2'] = qdf['FEEDBACK_SELECTWHEN'].str.contains('2')
qdf['FEEDBACK_SELECTWHEN_3'] = qdf['FEEDBACK_SELECTWHEN'].str.contains('3')
qdf['FEEDBACK_SELECTWHEN_4'] = qdf['FEEDBACK_SELECTWHEN'].str.contains('4')
qdf['FEEDBACK_SELECTWHEN_5'] = qdf['FEEDBACK_SELECTWHEN'].str.contains('5')
qdf['FEEDBACK_SELECTWHEN_6'] = qdf['FEEDBACK_SELECTWHEN'].str.contains('6')
qdf['FEEDBACK_SELECTWHEN_7'] = qdf['FEEDBACK_SELECTWHEN'].str.contains('7')
qdf['FEEDBACK_SELECTWHEN_8'] = qdf['FEEDBACK_SELECTWHEN'].str.contains('8')

In [33]:
qdf.to_csv("qualtrics_splitwhen_hri2023.csv")